# Quantum Annealing at Finite Temperature

# Quantum Approximate Thermalization
Code inspired by the implementation of [QABoM in PyQuil](https://github.com/MichaelBroughton/QABoM/blob/master/qRBM_final.py) by the authors of the paper. [Those slides](https://www.dropbox.com/s/16n5xnqcmsbktfk/LANL_QABoM.pdf?dl=0) by Guillaume Verdon were also particularly useful in understanding the algorithm. 

## Classical Boltzmann Machine

In a classical Restricted Boltzmann Machine (RBM), the goal is to generate samples from a probability distributition $P(\textbf{v})$ infered from the data, where $\textbf{v} \in \{0,1\}^n$. The assumption is that this distribution lies on a latent space that can be paramerized by a set of hidden variables $\textbf{h} \in \{0,1\}^n$, such that $P(\textbf{v})=\sum_hP(\textbf{v}|\textbf{h})P(\textbf{h})$. The joint probability is modeled as a Gibbs distribution with the energy defined by an Ising Model: $P(\textbf{v}, \textbf{h})=\frac{1}{Z} e^{-\beta E(\textbf{h},\textbf{v})}$ where Z is a normalization constant (called partition function) and $E(\textbf{h},\textbf{v})=-\sum_{i,j} W_{ij} h_i v_j$. It can then be shown that $p(\textbf{h}|\textbf{v})=\sigma(W \cdot \textbf{v})$ and $p(\textbf{v}|\textbf{h})=\sigma(W \cdot \textbf{h})$, where $\sigma$ is the sigmoid function defined by $\sigma(x)=\frac{1}{1+e^{-x}}$.

The training procedure consists in finding the weights $W$ that maximizes the log-likelihood $L=\sum_{\textbf{v} \in D} \log(p(\textbf{v}|W))$, where $D$ is a set of real observations (dataset). This function is differentiable and can be optimized using regular gradient ascent: $W_{ij}^{(t+1)}=W_{ij}^{(t)} + \eta \frac{\partial L}{\partial W_{ij}}$. Computing the gradient $\frac{\partial L}{\partial W_{ij}}$ is the hard part. Indeed, one can show that 

$$\frac{\partial L}{\partial W_{ij}}=\frac{1}{|D|} \sum_{\textbf{v} \in D} \mathbb{E}_{\textbf{h} \sim P(\textbf{h}|\textbf{v})}[h_i v_j] - \mathbb{E}_{(\textbf{h},\textbf{v}) \sim P(\textbf{h},\textbf{v})}[h_i v_j]$$.

The first expectancy (under the sum) is easy to compute and can be shown to be equal to $\sigma \left( \sum_j W_{ij} v_j \right) v_j$. One then just needs to sum those expectancy over the dataset.

However, the second expectancy cannot be simplified as easily, since the sum is over all possible $\textbf{v}$ and $\textbf{h}$. It would take an exponential amount of time to compute it exactly.

Classical RBM uses Monte-Carlo algorithms to approximate this expectancy.

## Quantum Boltzmann Machine

However, sampling is a major strength of quantum computers, and it should be possible to approximate the second expectancy with a quantum algorithm. For that, we start by quantizing the RBM model:

\begin{align}
E(\textbf{h},\textbf{v}) = -\sum_{i,j} W_{ij} h_i v_j & \longleftrightarrow H = -\sum_{<i,j>} W_{ij} \sigma^Z_i \sigma^Z_j \\
P(\textbf{v}, \textbf{h})=\frac{1}{Z} e^{-\beta E(\textbf{h},\textbf{v})} & \longleftrightarrow \rho = \frac{1}{Z} e^{-\beta H} \\
& ...
\end{align}

Concerning the gradient, the first expectancy only depends on the real data and can still be computed classically. However, the second one can now be computed in a quantum way:

$$\mathbb{E}_{(\textbf{h},\textbf{v}) \sim P(\textbf{h},\textbf{v})}[h_i v_j] \longleftrightarrow \textrm{tr}\left[\rho \sigma^Z_i \sigma^Z_j \right]$$

Computing this expectancy then consists in two steps:
* Preparing $\rho$ to approximate the thermal state of $H(W_{ij})$. If we new that the thermal state was a qubit $\psi$ (which means $\rho=|\psi \rangle \langle \psi |$), we could do apply a standard QAOA algorithm, starting from the ground state of $e^{-\beta H_m}$. The trick here is to purify $\rho$. If we call $\mathcal{H_1}$ our current Hilbert space (of dimension $n_{visible} + n_{hidden}$), purifying a density matrix $\rho$ consists in finding a second Hilbert space $\mathcal{H_2}$ such that there exists $| \psi \rangle \in \mathcal{H_1} \otimes \mathcal{H_2}$ such that $\rho = \textrm{tr}_{\mathcal{H_2}} \left( |\psi \rangle \langle \psi | \right)$. It can be shown that $| \psi \rangle =\sqrt{2 \cosh \beta} \sum_{z \in {-1,1}} e^{-\beta z} |z \rangle_{\mathcal{H_1}} \otimes | z \rangle_{\mathcal{H_2}}$ purifies $\rho=e^{-\beta H_m}$. This state can be built with a circuit composed uniquely of RX gates and CNOT gates
* Tracing over $\rho \sigma^Z_i \sigma^Z_j$ to obtain the expectancy. This can be done using the function `eval` of Qiskit Aqua.

In [1]:
import numpy as np

from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.wrapper import execute as q_execute
from qiskit.tools.qi.pauli import Pauli
from qiskit.tools.apps.optimization import eval_hamiltonian 
from qiskit_aqua.operator import Operator
from qiskit_aqua import get_initial_state_instance
from qiskit import Aer
from qiskit.tools.qi.qi import state_fidelity

from scipy.optimize import minimize
from functools import reduce
import itertools

from qaoa import QAOA

np.set_printoptions(precision=2)
%load_ext autoreload
%autoreload 2

## Parameters

### Network

We will use a circuit with $n_{visible} + n_{hidden}$ qubits, where the first $n_{visible}$ qubits represent the visible nodes and the $n_{hidden}$ last qubits represent the hidden nodes.

In [2]:
n_visible = 2
n_hidden = 1
n_qubits = n_visible + n_hidden
n_system = n_qubits*2 # due to environment qubits, used for purification

In [3]:
beta = 1 # inverse temperature

We initialize the weights with a Gaussian distribution of variance $\frac{2}{n_{hidden} + n_{visible}}$ (the so-called [Xavier initialization](http://andyljones.tumblr.com/post/110998971763/an-explanation-of-xavier-initialization), that guarantees good properties)

In [4]:
weights_init = np.random.normal(0, 2 / (n_hidden + n_visible), size=(n_visible, n_hidden))

### QAOA

In [5]:
n_gamma_nu = 1
nu_init = np.random.uniform(0, np.pi*2, n_gamma_nu) # initial nu values
gamma_init = np.random.uniform(0, np.pi*2, n_gamma_nu) # initial gamma values

### Data
Very simple dataset, same as in the original code. Rows are samples and columns features

In [21]:
# X_train = np.array([[1,1,-1,-1], [1,1,-1,-1], [-1,-1,1,1], [-1,-1,1,1]])
X_train = np.array([[-1, 1], [1,-1], [-1,1], [1,-1]])

### Optimization

In [22]:
n_epochs = 2
lr = 0.1

## Preparing the hamiltonians

In [23]:
def pauli_z(qubit, coeff):
    eye = np.eye((n_system))
    return Operator([[coeff, Pauli(eye[qubit], np.zeros(n_system))]])

def pauli_x(qubit, coeff):
    eye = np.eye((n_system))
    return Operator([[1, Pauli(np.zeros(n_system), eye[qubit])]])

def product_pauli_z(q1, q2, coeff):
    eye = np.eye((n_system))
    return Operator([[coeff, Pauli(eye[q1], np.zeros(n_system)) * Pauli(eye[q2], np.zeros(n_system))]])

### Mixer hamiltonian
Reference hamiltonian for QAOA, defined by $H_m = \sum_i^n \sigma^X_i$

In [24]:
Hm = reduce(lambda x,y:x+y,
            [pauli_x(i, 1) 
             for i in range(n_qubits)])
Hm.to_matrix()

### Cost hamiltonian

In [25]:
def ising_hamiltonian(weights):
    H = reduce(lambda x,y:x+y,
            [product_pauli_z(i,n_visible + j, -weights[i,j]) # J[i,j] as a first parameter
             for i,j in itertools.product(range(n_visible), range(n_hidden))])
    H.to_matrix()
    return H

## Initial state preparation
We prepare the intial state $|\psi_0 \rangle = \sqrt{2 cosh(\beta)} \sum_{z \in {1, -1}} e^{-\beta z} | z \rangle_S \otimes | z \rangle_E$, with $E$ a temporary space used for purification purpose. It can be shown that tracing out this state over $E$ reproduces the state $\rho \propto e^{-\beta H_m} $

In [26]:
qr = QuantumRegister(n_qubits * 2)

In [27]:
circuit_init = QuantumCircuit(qr)

alpha = 2 * np.arctan(np.exp(- beta / 2))
for i in range(n_qubits):
    circuit_init.rx(alpha, qr[n_qubits+i])
    circuit_init.cx(qr[n_qubits+i], qr[i])

## Useful functions

In [28]:
def get_thermalized_state(weights, maxiter):
    Hc = ising_hamiltonian(weights)
    print("Begin QAOA...")
    qaoa = QAOA(mixer_hamiltonian=Hm,
                cost_hamiltonian=Hc,
                quantum_register=qr,
                circuit_init=circuit_init,
                n_gamma_nu=n_gamma_nu)
    
    circuit, result = qaoa.optimize(maxiter=maxiter)
    print("Results of QAOA", result)
    
    return circuit

In [29]:
def evaluate_hamiltonian(hamiltonian, circuit):
    return np.real(hamiltonian.eval("matrix", circuit, 'statevector_simulator')[0]) # the value should always be real in theory, but for numerical reasons the imaginary part can be a very small number
        

In [30]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [33]:
def get_hidden_units(visible):
    return sigmoid(np.dot(visible, weights))

## Training

In [31]:
circuit = circuit_init
weights = weights_init.copy()

In [38]:
for epoch in range(n_epochs):
    thermalized_state = get_thermalized_state(weights, maxiter=2)
    
    delta_2 = np.zeros((n_visible, n_hidden))
    for v in range(n_visible):
        for h in range(n_hidden):
            delta_2 = evaluate_hamiltonian(pauli_z(v, 1) * pauli_z(h, 1), thermalized_state)
            
    hidden_probs = sigmoid(np.dot(X_train, weights))
    delta_1 = np.dot(X_train.T, hidden_probs) / len(X_train)
    
    weights += lr * (delta_1 - delta_2)

Begin QAOA...
test
test_fin
test
test_fin
test
test_fin
test
test_fin
test
test_fin
Results of QAOA  final_simplex: (array([[2.74, 5.6 ],
       [2.61, 5.89],
       [2.74, 5.89]]), array([0.01, 0.07, 0.07]))
           fun: 0.005704832557674188
       message: 'Maximum number of iterations has been exceeded.'
          nfev: 5
           nit: 2
        status: 2
       success: False
             x: array([2.74, 5.6 ])
Begin QAOA...
test
test_fin
test
test_fin
test
test_fin
test
test_fin
test
test_fin
Results of QAOA  final_simplex: (array([[0.07, 3.94],
       [0.07, 3.84],
       [0.07, 3.84]]), array([9.93e-05, 4.09e-03, 4.09e-03]))
           fun: 9.93252125969903e-05
       message: 'Maximum number of iterations has been exceeded.'
          nfev: 5
           nit: 2
        status: 2
       success: False
             x: array([0.07, 3.94])


## Testing
If the training worked, we should have $P(\textbf{h}|\textbf{v}) \sim 1$ for $\textbf{v}\in D$. Let's try.

In [37]:
get_hidden_units(X_train[0])

array([0.65])